# Multi Layer Perceptron

![MLP](mlp.png)

The multi layer perceptron is shown above. This MLP has 3 inputs 4 neurons at first layer, 2 neurons at second layer, and two neurons at output layer with tanh activation functions. Please refer to Perceptron/Adaline section. The circles with b are representing the bias term. All the connections inside the network are weights that are the learnable parameters including bias weights. The single weight is represented as $w_{current,in}^{layer}$. In this representation the current is the index of the neuron in the layer, in is the index of previous neurron or bias, and lastly layer is the index of the layer. For example, the purple connection is $w_{3,1}^{0}$ since it is the weight of neuron 3 in the layer 0 and connected to previous neuron 1. Likewise green connection is $w_{1,4}^{1}$ since it is the weight of neuron 1 in the layer 1 and connected to previous neuron 4 or bias.

## Forward pass - Inference

The forward pass through the multilayer perceptron or the term commonly used inference thorugh the network can be define with these three equations.

![Layer0](layer0.png)

![Layer1](layer1.png)

![Layer2-Output](layer2.png)

In these equations $v_{neuron}^{layer}$ is the linear combination, and the $y_{neuron}^{layer}$ is the output or the activation of the neuron. In order to keep consistency, tanh is used activation function. 

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import random

input_dimension = 3
input_data = np.random.rand(input_dimension + 1, 1) #bias added
input_data[-1] = 1

layer_0 = 4
layer_1 = 2
layer_out = 2

ground_truth = np.random.rand(layer_out, 1)

#Keep in mind the bias weights, all the weights are sampled from uniform distribution between 0 and 1. The substraction makes it -0.5 and 0.5.
weights_0 = np.random.rand(layer_0, input_dimension + 1) - 0.5
weights_1 = np.random.rand(layer_1, layer_0 + 1) - 0.5
weights_out = np.random.rand(layer_out, layer_1 + 1) - 0.5

print(weights_0, weights_0.shape)
print(weights_1, weights_1.shape)
print(weights_out, weights_out.shape)


[[-0.17822729  0.04863693  0.38278317  0.47639535]
 [-0.35616279  0.09280376 -0.08697504 -0.45763078]
 [-0.47053905  0.06403546 -0.10600353  0.36521369]
 [ 0.36463314 -0.41105599 -0.38549603 -0.26486601]] (4, 4)
[[-0.13255122 -0.27607883  0.0171619   0.48017992  0.34596086]
 [ 0.00691831  0.3529409   0.33376724  0.19320881  0.2830946 ]] (2, 5)
[[-0.03046202  0.33256503 -0.43053093]
 [ 0.42146095 -0.24667624  0.14493803]] (2, 3)


In [3]:
#Now lets use inference to calculate outputs
input_0 = input_data
print(input_0.shape)
lin_comb_0 = np.matmul(weights_0, input_0) #v
output_0 = np.tanh(lin_comb_0) #y

#Add bias to output 0
input_1 = np.concatenate((output_0, np.ones((1,1))))
print(input_1.shape)

lin_comb_1 = np.matmul(weights_1, input_1) #v
output_1 = np.tanh(lin_comb_1) #y

#Add bias to output 1
input_out = np.concatenate((output_1, np.ones((1,1))))
print(input_out.shape)

lin_comb_out = np.matmul(weights_out, input_out) #v
output_out = np.tanh(lin_comb_out) #y
print(output_out.shape)

(4, 1)
(5, 1)
(3, 1)
(2, 1)


## Training

### Mean Squared Error

In training phase, weights are updated by using backpropagation algorithm. The mean squared error, between output of the adaline $y$ and the ground truth $y_d$, is calculated by using
$$mse=\frac{1}{n}\overset{n}{\sum_{j}}(y_{dj}-y_j)^2$$
In this equation $n$ is the number of outputs.

In [4]:
def mean_square_error(ground_truth, desired):
    temp = np.square(ground_truth - desired)
    temp = np.mean( temp )
    return temp

print("Mean squared error between {} and {} is {}".format(ground_truth, output_out, mean_square_error(ground_truth, output_out)))

Mean squared error between [[0.74470221]
 [0.91208701]] and [[-0.4089246 ]
 [ 0.18176664]] is 0.9321113316914262


### Backpropagation

#### Output Layer

IMPORTAN NOTE: THE DERIVATIVES ARE FOR DEMONSTRATION ONLY, THEY ARE NOT ALLIGNED INITIALLY, AFTERWARDS THE MATRIX MULTIPLICATIONS AND ELEMENT-WISE OPERATIONS ARE EXPLAINED IN DETAIL. 
The ground truth is defined as the vector $y_d$, then the error can be defined as
$$ e = \begin{bmatrix} e_0 \\ e_1 \end{bmatrix} = \begin{bmatrix} y_{d,0} - y_0 \\ y_{d,1} - y_1 \end{bmatrix}$$

Becareful that the $E$ represents the means squared error.  
Then the backpropagation term for the weights in output layer can be defined as

$$\frac{\partial E}{\partial w^{out}(k)}=\frac{\partial E}{\partial e}\frac{\partial e}{\partial y^{out}}\frac{\partial y^{out}}{\partial v^{out}}\frac{\partial v^{out}}{\partial w^{out}(k)}$$
Each derivative term is defined as,  
*Derivative of MSE: $E=\frac{1}{2}e^2$ $\frac{\partial E}{\partial e} = e$

*Derivative of error term: $\frac{\partial e}{\partial y^{out}} = -1$

*Derivative of activation function: $\frac{\partial y^{out}}{\partial v^{out}} = 1 - tanh^2(v^{out}) = 1 - (y^{out})^2$

*Derivative of linear combination: $\frac{\partial v^{out}}{\partial w^{out}(k)} = x^{out}$

The $x^{out}$ is the output of the previous layer with bias = 1 concatanated to the end of the vector.

$$\frac{\partial E}{\partial w^{out}(k)}= e\circ(-1)\circ(1 - (y^{out})^2)\times (x^{out})^T$$

$$\frac{\partial E}{\partial w^{out}(k)}= \begin{bmatrix} e_0 \\ e_1 \end {bmatrix}\circ (-1)\circ (1 - (\begin{bmatrix} y^{out}_0 \\ y^{out}_1 \end{bmatrix})^2) \times \begin{bmatrix} y^{1}_0 & y^{1}_1 & 1 \end{bmatrix} $$

The $e$ and $y^{out}$ multiplitaions is element-wise and the last one is matrix multiplitaion.

In [5]:
mse = mean_square_error(ground_truth, output_out)
error = ground_truth - output_out 

temp = error * -1 * (1 - output_out**2)
print(temp.shape)
print(input_out.shape)
output_derivative = np.matmul(temp.reshape(-1, 1), input_out.reshape(1, -1))
print(output_derivative.shape)

(2, 1)
(3, 1)
(2, 3)


#### Hidden Layers

In order compute the gradients of hidden layers, the local gradients term is come into play. If you check the multilayer perceptron schematic the red line shows a single way of data flow through MLP. Calculation of output layer only requires information from the error but to calculate weights between layer 1 and layer 0, $w^1$ now the both output neurons effect the outcome. So that change in these neurons must back propagated.

Let's look at the derivative calculation, this calculation is demonstration only go below for matrix operations.

$$\frac{\partial E}{\partial w^{1}(k)}=\frac{\partial E}{\partial e}\frac{\partial e}{\partial y^{out}}\frac{\partial y^{out}}{\partial v^{out}}\frac{\partial v^{out}}{\partial y^{1}}\frac{\partial y^{1}}{\partial v^{1}}\frac{\partial v^{1}}{\partial w^{1}(k)}$$

The strange thing here is the $\frac{\partial v^{out}}{\partial y^{1}}$ term. The meaning of this term is how much linear combination of output neurons change with the inputs. The most important thing to understand is that the inputs of the output layer is the outcome of the previous layer. Then normal back propagation can continue. The terms are defined as

* Local gradient: $\delta^{out} = \frac{\partial E}{\partial e}\frac{\partial e}{\partial y^{out}}\frac{\partial y^{out}}{\partial v^{out}}$

* Derivative of linear combination of output layer $\frac{\partial v^{out}}{\partial y^{1}} = w^{out}$

* Derivative of activation function: $\frac{\partial y^{1}}{\partial v^{1}} = 1 - tanh^2(v^{1}) = 1 - (y^{1})^2$

* Derivative of hidden layer linear combination: $\frac{\partial v^{1}}{\partial w^{1}(k)} = x^{1}$

The derivative can be rewritten as, (demonstration only)

$$\frac{\partial E}{\partial w^{1}(k)}= \delta^{out} w^{out} (1 - (y^{1})^2) x^{1}$$

In this equation, the multiplications can cause confussion and wrong calculations. Keep in mind that we are trying to move gradients backward in the graph. So that every gradient should follow a route and multiplied by the weight of that route and accumulated. First let's show our terms and try to visualize on the graph bellow. 
$$ \delta^{out} = \begin{bmatrix} \delta^{out}_0 \\ \delta^{out}_1 \end {bmatrix} \quad w^{out} = \begin{bmatrix} w^{out}_{0,0} & w^{out}_{0,1} & w^{out}_{0,2} \\ w^{out}_{1,0} & w^{out}_{1,1} & w^{out}_{1,2} \end {bmatrix}  $$
Here bias terms are the weight with second underscript 2, let's remove them. Remember, that bias terms always 1, they are not effected by previous layers or do not effect the previous layers.
$$ \delta^{out} = \begin{bmatrix} \delta^{out}_0 \\ \delta^{out}_1 \end {bmatrix} \quad w^{out} = \begin{bmatrix} w^{out}_{0,0} & w^{out}_{0,1} \\ w^{out}_{1,0} & w^{out}_{1,1} \end {bmatrix}  $$

![Local Gradients](local_gradients.png)

From the graph above, the accumulation of the weight multiplied local gradients can be seen. So the equation becomes,

$$ (w^{out})^T \delta^{out} = \begin{bmatrix} w^{out}_{0,0} & w^{out}_{1,0} \\ w^{out}_{0,1} & w^{out}_{1,1} \end {bmatrix} \begin{bmatrix} \delta^{out}_0 \\ \delta^{out}_1 \end {bmatrix} = \begin{bmatrix} w^{out}_{0,0}\delta^{out}_0 + w^{out}_{1,0}\delta^{out}_1\\ w^{out}_{0,1}\delta^{out}_0 + w^{out}_{1,1}\delta^{out}_1\ \end {bmatrix} $$

Now let's add the derivative of the activation function. This term is the local gradient of layer 1.

$$ \delta^{1} = (w^{out})^T \times \delta^{out}\circ (1 - (y^{1})^2)$$

Respectively, the local gradient of layer 0 is,

$$ \delta^{0} = (w^{1})^T \times \delta^{1}\circ (1 - (y^{0})^2)$$

The activation function elements are element-wise multiplication.

Finally derivatives are defines as,

$$\frac{\partial E}{\partial w^{out}(k)}= \delta^{out} \times (x^{out})^T \quad \frac{\partial E}{\partial w^{1}(k)}= \delta^{1} \times (x^{1})^T \quad \frac{\partial E}{\partial w^{0}(k)}= \delta^{0} \times (x^{0})^T $$

Keep in mind that bias terms are removed when calculating local gradients and $x^0$is the input of the neural network. Now the partial derivatives are computed by using local gradients.


In [6]:
local_gradient_out = error * -1 * (1 - output_out**2)

weight_out_temp = np.transpose(weights_out[:, :-1]) #no bias and transpose
print(weight_out_temp.shape, local_gradient_out.shape)
temp1 = np.matmul(weight_out_temp, local_gradient_out)
print(temp1.shape)
local_gradient_1 = temp1 * (1 - output_1**2)

weight_1_temp = np.transpose(weights_1[:, :-1]) #no bias and transpose
print(weight_1_temp.shape, local_gradient_1.shape)
temp0 = np.matmul(weight_1_temp, local_gradient_1)
print(temp0.shape)
local_gradient_0 = temp0 * (1 - output_0**2)
print(local_gradient_0.shape)

(2, 2) (2, 1)
(2, 1)
(4, 2) (2, 1)
(4, 1)
(4, 1)


In [7]:
print(local_gradient_out.shape, input_out.shape)
derivative_out = np.matmul(local_gradient_out, input_out.transpose())
print(derivative_out.shape)

print(local_gradient_1.shape, input_1.shape)
derivative_1 = np.matmul(local_gradient_1, input_1.transpose())
print(derivative_1.shape)

print(local_gradient_0.shape, input_0.shape)
derivative_0 = np.matmul(local_gradient_0, input_0.transpose())
print(derivative_0.shape)

(2, 1) (3, 1)
(2, 3)
(2, 1) (5, 1)
(2, 5)
(4, 1) (4, 1)
(4, 4)


### Weight Update

The weights can be updated by using the equaion,
$$w(k+1) = w(k) - lr \frac{\partial E}{\partial w(k)}$$
$lr$ is learning rate.

In [10]:
lr = 1e-3
weights_0 = weights_0 - lr * derivative_0
weights_1 = weights_1 - lr * derivative_1
weights_out = weights_out - lr * derivative_out